In [1]:
######################################################################
# Filename:    GFS_IWT.py
# Author:      Sawyer Smith sas053@ucsd.edu modified code provided by Deanna Nash dnash@ucsd.edu
# Description: Script to take downloaded GFS u, v, qvapor, qcloud, qrain, qice, qsnow data for each given init date and lead time, 
#              and compute the IVT, IWT and ICT/IVT ratio

######################################################################

import itertools ## need this for the cbarticks
import datetime


## import libraries
import os, sys
import yaml
import xarray as xr
import numpy as np
import pandas as pd
import shutil
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## import plotting modules
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import cartopy.crs as ccrs
from matplotlib.colorbar import Colorbar # different way to handle colorbar
import domains
from plotter import draw_basemap
import cw3ecmaps as ccmaps
import re  # Regular expressions


In [2]:
def calc_transport_manual(ds, include_condensates=False):
    '''
    Calculates vertically integrated vapor or water transport (IVT or IWT).
    
    Parameters:
    -----------
    ds : xarray.Dataset
        Must include variables: q, u, v
        Optional: rwmr, clwmr, icmr, snmr (if include_condensates=True)
    
    include_condensates : bool
        If True, includes condensate mixing ratios in the total mixing ratio.

    Returns:
    --------
    xarray.Dataset with ivtu, ivtv, and magnitude (ivt or iwt)
    '''

    pressure = ds.isobaricInhPa.values * 100  # convert hPa to Pa
    dp = np.diff(pressure)
    g = 9.81  # gravity

    qu_lst, qv_lst = [], []

    # enumerate through pressure levels so we select the layers
    for i, pres in enumerate(ds.isobaricInhPa.values[:-1]):
        pres2 = ds.isobaricInhPa.values[i+1]
        tmp = ds.sel(isobaricInhPa=[pres, pres2]) # select layer
        tmp = tmp.mean(dim='isobaricInhPa', skipna=True) # average q, u, v in layer
        # calculate ivtu in layer
        qu = ((tmp.q*tmp.u*dp[i])/g)*-1
        qu_lst.append(qu)
        # calculate ivtv in layer
        qv = ((tmp.q*tmp.v*dp[i])/g)*-1
        qv_lst.append(qv)

    ## add up u component of ivt from each layer
    qu = xr.concat(qu_lst, pd.Index(pressure[:-1], name="pres"))
    qu = qu.sum('pres')
    qu.name = 'ivtu' if not include_condensates else 'iwtu'
    
    # ## add up v component of ivt from each layer
    qv = xr.concat(qv_lst, pd.Index(pressure[:-1], name="pres"))
    qv = qv.sum('pres')
    qv.name = 'ivtv' if not include_condensates else 'iwtv'
    
    ## calculate IVT magnitude
    ivt = np.sqrt(qu**2 + qv**2)
    ivt.name = 'ivt' if not include_condensates else 'iwt'

    ds = xr.merge([qu, qv, ivt])
    
    return ds

In [3]:
def plot_transport(ds, mag_name, init_date, domain_name='intwest',
                    varname='ivt', fmt='png', fname_prefix='IVT'):

    # === Setup
    datacrs = ccrs.PlateCarree()
    mapcrs = domains.extent[domain_name]['ccrs']
    ext = domains.extent[domain_name]['ext']
    dx = domains.extent[domain_name]['xticks']
    dy = domains.extent[domain_name]['yticks']
    figsize = domains.extent[domain_name]['figsize']

    nrows, ncols = 1, 2  # main + colorbar

    fig = plt.figure(figsize=figsize)
    fig.dpi = 300

    fname = f'{fname_prefix}_{domain_name}_{init_date}'

    gs = GridSpec(nrows, ncols, height_ratios=[1], width_ratios=[1, 0.05],
                  wspace=0.025, hspace=0.05)

    # === Basemap
    ax = fig.add_subplot(gs[0, 0], projection=mapcrs)
    ax = draw_basemap(ax, extent=ext, xticks=dx, yticks=dy,
                      grid=True, left_lats=True, right_lats=False)
    ax.set_extent(ext, datacrs)

    # === Colormap and normalization
    if (varname == 'ivt'):
        cmap, norm, bnds, cbarticks, cbarlbl = ccmaps.cmap('ivt_intwest')
    elif (varname == 'iwt'):
        cmap, norm, bnds, cbarticks, cbarlbl = ccmaps.cmap('iwt_intwest')
    elif (varname == 'ict'):
        cmap, norm, bnds, cbarticks, cbarlbl = ccmaps.cmap('ict_intwest')
    elif (varname == 'ratio'):
        cmap, norm, bnds, cbarticks, cbarlbl = ccmaps.cmap('ratio_intwest')
        

    # === Plot data
    lons = ds.longitude
    lats = ds.latitude
    data = ds[mag_name]

    cf = ax.contourf(lons, lats, data,
                     transform=datacrs, cmap=cmap, norm=norm, levels=bnds, alpha=0.9)

    # === Colorbar
    cbax = fig.add_subplot(gs[0, -1])
    cbarticks = list(itertools.compress(bnds, cbarticks))
    cb = Colorbar(ax=cbax, mappable=cf, orientation='vertical',
                  ticklocation='right', ticks=cbarticks)
    cb.set_label(cbarlbl, fontsize=11)
    cb.ax.tick_params(labelsize=12)

    # === Title
    if 'init_date' in ds.attrs and 'valid_date' in ds.attrs:
        init_dt = pd.to_datetime(ds.attrs['init_date'], format="%Y%m%d_%H")
        valid_dt = pd.to_datetime(ds.attrs['valid_date'], format="%Y%m%d_%H")
        lead_hours = int((valid_dt - init_dt).total_seconds() / 3600)
        lead_tag = f"f{lead_hours:03d}"  # → f006, f012
        t = f"Init: {init_dt.strftime('%Y-%m-%d %H:%M')} | Valid: {valid_dt.strftime('%Y-%m-%d %H:%M')}"

    else:
        t = "Unknown Initialization Time"
    
    if fname_prefix=="RATIO":
        ax.set_title(f'ICT/IVT {fname_prefix} | {t}', loc='left')
    else:
        ax.set_title(f'{fname_prefix} | {t}', loc='left')
    
    # === Save figure to auto-created folder based on init time
    if 'init_date' in ds.attrs:
        output_root = '/home/sas042/jupyter_notebooks/IWT/GFS/figs/'
        date_tag = ds.attrs['init_date'][:8]  # YYYYMMDD
        output_dir = os.path.join(output_root, f'{date_tag}_GFS')
        os.makedirs(output_dir, exist_ok=True)
    
        save_path = os.path.join(output_dir, f'{lead_tag}_{fname}.{fmt}')
        fig.savefig(save_path, bbox_inches='tight', dpi=fig.dpi)
        print(f"Saved: {save_path}")
    else:
        fig.savefig(f'{fname}.{fmt}', bbox_inches='tight', dpi=fig.dpi)
        print(f"Saved without organized folder: {fname}.{fmt}")
    
    plt.close(fig)




In [4]:

# Forecast hours you want to loop through
forecast_hours = ['00', '06', '12']

# Base directory where all GFS folders are
base_dir = '/home/sas042/jupyter_notebooks/IWT/GFS'

# Loop through each GFS initialization folder
for date_folder in sorted(os.listdir(base_dir)):
    full_path = os.path.join(base_dir, date_folder)
    print(full_path)
    # Skip non-date folders (e.g., ".ipynb_checkpoints")
    if not os.path.isdir(full_path) or not re.fullmatch(r'\d{8}', date_folder):
        continue  # Skip anything that isn't a directory

    year = date_folder[:4]
    month = date_folder[4:6]
    day = date_folder[6:8]
    
    init_str = f'{year}{month}{day}_00'  # Initialization at 00Z
    init_dt = pd.to_datetime(init_str, format="%Y%m%d_%H")
    
    for hour in forecast_hours:
        fname = os.path.join(full_path, f'gfs.t00z.pgrb2.0p25.f0{hour}')
        if not os.path.exists(fname):
            print(f"Missing: {fname}")
            continue

        print(f"Processing: {fname} for GFS fcst initialized on {date_folder}")
        valid_dt = init_dt + pd.to_timedelta(int(hour), unit='h')
        valid_str = valid_dt.strftime("%Y%m%d_%H")

        print(f"Plotting init={init_str}, valid={valid_str}")
        
        vardict = {
               
                "rwmr":{'typeOfLevel': 'isobaricInhPa', 'shortName': 'rwmr'}, # rain mixing ratio (kg/kg)
                "clwmr":{'typeOfLevel': 'isobaricInhPa', 'shortName': 'clwmr'}, # cloud water mixing ratio (kg/kg)
                "icmr":{'typeOfLevel': 'isobaricInhPa', 'shortName': 'icmr'}, # ice mixing ratio (kg/kg)
                "snmr":{'typeOfLevel': 'isobaricInhPa', 'shortName': 'snmr'},  # snow mixing ratio (kg/kg)
                "u_wind":{"shortName":'u'}, #U-component of wind
                "v_wind":{"shortName":'v'}, #V-component of wind
                "temperature":{"shortName":'t'}, #Temperature
                "specific_humidity":{"shortName":'q'}, #Specific humidity
                "sfcp": {'typeOfLevel': 'surface',  'level': 0, 'paramId': 134, 'shortName': 'sp'} # surface pressure
                }
        
        # Mixing ratio variables
        ds_rwmr = xr.open_dataset(fname, engine='cfgrib',
            backend_kwargs={'filter_by_keys': {'typeOfLevel': 'isobaricInhPa', 'shortName': 'rwmr'}}) # rain mixing ratio (kg/kg)
        ds_clwmr = xr.open_dataset(fname, engine='cfgrib',
            backend_kwargs={'filter_by_keys': {'typeOfLevel': 'isobaricInhPa', 'shortName': 'clwmr'}}) # cloud water mixing ratio (kg/kg)
        ds_icmr = xr.open_dataset(fname, engine='cfgrib',
            backend_kwargs={'filter_by_keys': {'typeOfLevel': 'isobaricInhPa', 'shortName': 'icmr'}}) # ice mixing ratio (kg/kg)
        ds_snmr = xr.open_dataset(fname, engine='cfgrib',
            backend_kwargs={'filter_by_keys': {'typeOfLevel': 'isobaricInhPa', 'shortName': 'snmr'}}) # snow mixing ratio (kg/kg)
        
        # Other key variables (u, v, q, t)
        ds_u = xr.open_dataset(fname, engine='cfgrib',
            backend_kwargs={'filter_by_keys': {'typeOfLevel': 'isobaricInhPa', 'shortName': 'u'}})
        ds_v = xr.open_dataset(fname, engine='cfgrib',
            backend_kwargs={'filter_by_keys': {'typeOfLevel': 'isobaricInhPa', 'shortName': 'v'}})
        ds_q = xr.open_dataset(fname, engine='cfgrib',
            backend_kwargs={'filter_by_keys': {'typeOfLevel': 'isobaricInhPa', 'shortName': 'q'}})
        ds_t = xr.open_dataset(fname, engine='cfgrib',
            backend_kwargs={'filter_by_keys': {'typeOfLevel': 'isobaricInhPa', 'shortName': 't'}})
        ds_sfcp = xr.open_dataset(fname, engine='cfgrib',
            backend_kwargs={'filter_by_keys': {'typeOfLevel': 'surface',  'level': 0, 'paramId': 134, 'shortName': 'sp'}})
        
        # Only keep the levels we want for the integrated flux calculations
        
        lvl_thres = 200
        ds_rwmr = ds_rwmr.sel(isobaricInhPa=ds_rwmr.isobaricInhPa[ds_rwmr.isobaricInhPa >= lvl_thres])
        ds_clwmr = ds_clwmr.sel(isobaricInhPa=ds_clwmr.isobaricInhPa[ds_clwmr.isobaricInhPa >= lvl_thres])
        ds_icmr  = ds_icmr.sel(isobaricInhPa=ds_icmr.isobaricInhPa[ds_icmr.isobaricInhPa >= lvl_thres])
        ds_snmr  = ds_snmr.sel(isobaricInhPa=ds_snmr.isobaricInhPa[ds_snmr.isobaricInhPa >= lvl_thres])
        
        ds_u = ds_u.sel(isobaricInhPa=ds_u.isobaricInhPa[ds_u.isobaricInhPa >= lvl_thres])
        ds_v = ds_v.sel(isobaricInhPa=ds_v.isobaricInhPa[ds_v.isobaricInhPa >= lvl_thres])
        ds_q = ds_q.sel(isobaricInhPa=ds_q.isobaricInhPa[ds_q.isobaricInhPa >= lvl_thres])
        ds_t = ds_t.sel(isobaricInhPa=ds_t.isobaricInhPa[ds_t.isobaricInhPa >= lvl_thres])
        
        # Extract the individual DataArrays
        q     = ds_q.q
        rwmr  = ds_rwmr.rwmr
        clwmr = ds_clwmr.clwmr
        icmr  = ds_icmr.icmr
        snmr  = ds_snmr.snmr
        
        # Sum them
        twmr = q + rwmr + clwmr + icmr + snmr
        twmr.name = 'q'  # Total water mixing ratio but retaining short name as 'q' for compatibility in calc_transport_manual() function 
        
        # Optionally assign attributes
        twmr.attrs['long_name'] = 'Total water mixing ratio (vapor + hydrometeors)'
        twmr.attrs['units'] = 'kg kg-1'
        
        ds_twmr = xr.merge([ds_u, ds_v, twmr, ds_t])
        ds_ivt = xr.merge([ds_u, ds_v, ds_q, ds_t])
        
        ivt_ds = calc_transport_manual(ds_ivt, include_condensates=False)
        iwt_ds = calc_transport_manual(ds_twmr, include_condensates=True)
        
        ict = iwt_ds['iwt'] - ivt_ds['ivt']
        ict.name = 'ict'
        ict_ds = xr.Dataset({'ict': ict})
        
        # Compute ICT/IVT ratio expressed as a percent (%)
        ratio = (ict_ds['ict']/ivt_ds['ivt']) * 100
        ratio.name = 'ratio'
        ratio_ds = xr.Dataset({'ratio': ratio})
        
        # Attach metadata used by plot_transport:
        for ds in [ivt_ds, iwt_ds, ict_ds, ratio_ds]:
            ds.attrs['init_date'] = init_str
            ds.attrs['valid_date'] = valid_str
            
        ## Plot IVT, IWT, ICT, and ICT/IVT ratio from GFS data
        init_date = f'{year}{month}{day}_{hour}'
        
        plot_transport(ivt_ds, 'ivt', varname='ivt', init_date=init_date)
        plot_transport(iwt_ds, 'iwt', varname='iwt', fname_prefix='IWT', init_date=init_date)
        plot_transport(ict_ds,'ict',varname='ict', fname_prefix='ICT', init_date=init_date, domain_name='intwest')
        plot_transport(ratio_ds,'ratio',varname='ratio', fname_prefix='RATIO', init_date=init_date, domain_name='intwest')

/home/sas042/jupyter_notebooks/IWT/GFS/.ipynb_checkpoints
/home/sas042/jupyter_notebooks/IWT/GFS/20241218
Processing: /home/sas042/jupyter_notebooks/IWT/GFS/20241218/gfs.t00z.pgrb2.0p25.f000 for GFS fcst initialized on 20241218
Plotting init=20241218_00, valid=20241218_00
Saved: /home/sas042/jupyter_notebooks/IWT/GFS/figs/20241218_GFS/f000_IVT_intwest_20241218_00.png
Saved: /home/sas042/jupyter_notebooks/IWT/GFS/figs/20241218_GFS/f000_IWT_intwest_20241218_00.png
Saved: /home/sas042/jupyter_notebooks/IWT/GFS/figs/20241218_GFS/f000_ICT_intwest_20241218_00.png
Saved: /home/sas042/jupyter_notebooks/IWT/GFS/figs/20241218_GFS/f000_RATIO_intwest_20241218_00.png
Processing: /home/sas042/jupyter_notebooks/IWT/GFS/20241218/gfs.t00z.pgrb2.0p25.f006 for GFS fcst initialized on 20241218
Plotting init=20241218_00, valid=20241218_06
Saved: /home/sas042/jupyter_notebooks/IWT/GFS/figs/20241218_GFS/f006_IVT_intwest_20241218_06.png
Saved: /home/sas042/jupyter_notebooks/IWT/GFS/figs/20241218_GFS/f006_IWT_